## Ejercicio 1
Considerando los logs de acciones realizadas sobre ítems, mostrar el top 10 de ids.

## Preparando sesión de Spark

### Spark session y Spark context

In [1]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Lectura de archivos

In [2]:
#import pandas as pd
import numpy as np 

In [3]:
#texts = pd.read_csv('./WikipediaDataset/logs.csv', usecols = ["action", "logtype", "title"])
#texts.to_parquet('./WikipediaDataset/logs_ActionLogtypeTitle.parquet')
#del texts
#Dataset de logs.parquet fue creado con las líneas anteriores, y ahora puede ser leído de la siguiente forma:
dataframeLogs = sqlContext.read.parquet('./WikipediaDataset/logs_ActionLogtypeTitle.parquet')
rddLogs = dataframeLogs.rdd

In [4]:
#texts = pd.read_csv('./WikipediaDataset/contents.csv', usecols = ["title", "id"])
#texts.to_parquet('./WikipediaDataset/contentsTitleId.parquet')
#del texts
dataframeContents = sqlContext.read.parquet('./WikipediaDataset/contentsTitleId.parquet')
rddContents = dataframeContents.rdd

## Preparando y limpiando los datasets

Archivo Logs

In [5]:
rddLogs

MapPartitionsRDD[7] at javaToPython at NativeMethodAccessorImpl.java:0

In [6]:
rddLogs1 = rddLogs.cache()

In [7]:
rddLogs1.take(5)

[Row(logtype='delete', action='delete', title='Verificacionismo'),
 Row(logtype='delete', action='delete', title='Rapunzel'),
 Row(logtype='delete', action='delete', title='Macromedia Flash'),
 Row(logtype='delete', action='delete', title='Archivo:Gnu head.png'),
 Row(logtype='delete', action='delete', title='Archivo:Gnu-head-sm.jpg')]

In [8]:
rddLogs1.count()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 35938)
Traceback (most recent call last):
  File "/home/andrezszambrano/Anaconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/andrezszambrano/Anaconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/andrezszambrano/Anaconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/andrezszambrano/Anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-258b6d6ad1c6>", line 1, in <module>
    rddLogs1.count()
  File "/home/andrezszambrano/spark-3.2.

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
rddLogs1 = rddLogs1.filter(lambda x: ((x[0] != None) and (x[1] != None) and (x[2] != None)))

In [ ]:
rddLogs.count()

In [ ]:
rddTitulos = rddLogs1.map(lambda x: (x[2], 1))
rddTitulos.take(5)

In [ ]:
rddTitulos = rddTitulos.reduceByKey(lambda x, y: x + y)
rddTitulos.take(5)

Ya el rddTitulos está listo para hacer un join.

In [ ]:
rddContents

In [ ]:
rddContents1 = rddContents.cache()

In [ ]:
rddContents1.take(5)

In [ ]:
rddContents1.count()

In [ ]:
rddContents1 = rddContents1.filter(lambda x: ((x[0] != None) and (x[1] != None)))
rddContents1.count()

In [ ]:
rddContents1 = rddContents.map(lambda x: (x[0], x[1]))
rddContents1.take(5)

## Desarrollo

Se procede a hacer un inner join para quedarse con la intersección de ambos rdd. Como tanto en el rdd de losgs y el rdd de contents no hay títulos repetidos, esta operación no dará filas con títulos repetidos. 

In [ ]:
rddJoin = rddTitulos1.join(rddContents1)
rddJoin.count()

In [ ]:
rddJoin.take(5)

Finalmente se pueden ordenar los contenidos según la cantidad de veces de que el título aparece en logs. Se procede entonces a quedarse con los 10 primeros que aparecen más en logs.

In [ ]:
listTop10 = rddJoin.takeOrdered(10, lambda x: -x[1][0]) 
print(listTop10)

## Conclusión
Se puede observar que los contenidos con más cantidad de logs están relacionados a los contenidos pertenecientes a las subcategorías de 'Wikipedia:', 'Anexo:'.  

## En limpio
A lo largo de la consigna se fue descubriendo los rdds para ir verificando cuáles operaciones hacer y que todo funcione correctamente. Con esto en mente está claro que la consigna puede ser resuelta con menos líneas de código, por lo que se procede a mostrar una solución 'final', con los rdds recién leídos y cargados, saltándose los pasos intermedios de investigación de los rdds.

Las siguientes transformaciones y acciones están hechas sobre los rdds originales obtenidos al hacer la lectura de los archivos correspondientes.

In [ ]:
rddTitulos = rddLogs.filter(lambda x: ((x[0] != None) and (x[1] != None) and (x[2] != None)))\
.map(lambda x: (x[2], 1)).reduceByKey(lambda x, y: x + y)

rddContents = rddContents.filter(lambda x: ((x[0] != None) and (x[1] != None))).map(lambda x: (x[0], x[1]))

In [ ]:
print(rddTitulos.join(rddContents).takeOrdered(10, lambda x: -x[1][0]))